# LlamaIndex Practice

Welcome to your hands-on practice with LlamaIndex! In this notebook, you'll learn how to:
- Create custom tools for AI agents
- Build a functional agent using Google Gemini
- Integrate Model Context Protocol (MCP) tools
- Implement observability and debugging
- Complete a creative challenge to apply your knowledge

## What You'll Build

In this notebook, you'll create an intelligent **Travel Planning Assistant** that can:
- Search for flight information
- Get weather forecasts for destinations
- Convert currencies for travel budgeting
- Use Model Context Protocol (MCP) for enhanced capabilities

By the end, you'll understand the complete agent creation flow with LlamaIndex and be ready to build your own creative agents!

## Step 1: Initial Setup & Installation

First, let's install the required packages. We need:
- **llama-index-llms-openai**: OpenAI integration
- **llama-index**: Core LlamaIndex framework
- **llama-index-tools-mcp**: Model Context Protocol tools support

In [ ]:
%pip install llama-index-llms-openai llama-index llama-index-tools-mcp -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.8/96.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 12.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, 

## Step 2: Configure API Key

You'll need an OpenAI API key to use GPT-4.1-mini. You also need to provide the base URL.

In [2]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key: ")

if "OPENAI_BASE_URL" not in os.environ:
    os.environ["OPENAI_BASE_URL"] = getpass.getpass("Enter your OpenAI Base URL: ")

Enter your OpenAI API Key: ··········
Enter your OpenAI Base URL: ··········


## Step 3: Create Custom Tools

Tools are functions that your agent can call to perform specific tasks. Let's create travel-related tools.

**Key Concepts:**
- Each tool is a Python function with clear docstrings
- Type hints help the LLM understand parameter types
- Docstrings describe what the tool does (the LLM reads this!)

In [3]:
from datetime import datetime, timedelta
import random


def search_flights(origin: str, destination: str, date: str) -> str:
    """Search for available flights between two cities on a specific date.

    Args:
        origin: The departure city (e.g., 'New York')
        destination: The arrival city (e.g., 'Paris')
        date: The travel date in YYYY-MM-DD format

    Returns:
        A string with flight information including price and duration
    """
    # Simulate flight search
    flight_number = f"FL{random.randint(100, 999)}"
    price = random.randint(300, 1500)
    duration = random.randint(4, 15)

    return f"Found flight {flight_number} from {origin} to {destination} on {date}. Price: ${price}, Duration: {duration}h"


def get_weather(city: str, days_ahead: int = 0) -> str:
    """Get weather forecast for a city.

    Args:
        city: The city name to get weather for
        days_ahead: Number of days ahead to forecast (0 = today, 1 = tomorrow, etc.)

    Returns:
        A string describing the weather conditions
    """
    conditions = ["Sunny", "Partly Cloudy", "Cloudy", "Rainy", "Stormy"]
    condition = random.choice(conditions)
    temp = random.randint(15, 30)
    forecast_date = datetime.now() + timedelta(days=days_ahead)

    return f"Weather in {city} on {forecast_date.strftime('%Y-%m-%d')}: {condition}, {temp}°C"


def convert_currency(amount: float, from_currency: str, to_currency: str) -> str:
    """Convert an amount from one currency to another.

    Args:
        amount: The amount to convert
        from_currency: Source currency code (e.g., 'USD')
        to_currency: Target currency code (e.g., 'EUR')

    Returns:
        A string with the converted amount
    """
    # Simplified exchange rates (in practice, you'd use a real API)
    rates = {
        ("USD", "EUR"): 0.92,
        ("USD", "GBP"): 0.79,
        ("USD", "JPY"): 149.50,
        ("EUR", "USD"): 1.09,
        ("GBP", "USD"): 1.27,
    }

    rate = rates.get((from_currency, to_currency), 1.0)
    converted = amount * rate

    return f"{amount} {from_currency} = {converted:.2f} {to_currency}"

## Step 4: Initialize the Language Model

Now let's set up Google Gemini as our LLM. We're using **gemini-2.5-flash** for fast, efficient responses.

In [4]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(
    model="gpt-4.1-mini",
    api_key=os.environ["OPENAI_API_KEY"],
    api_base=os.environ["OPENAI_BASE_URL"]
)

## Step 4: Initialize the Language Model

Now let's set up OpenAI GPT-4.1-mini as our LLM.

In [5]:
from llama_index.core.agent.workflow import FunctionAgent

agent = FunctionAgent(
    tools=[search_flights, get_weather, convert_currency],
    llm=llm,
    verbose=True  # This helps us see what the agent is doing
)

## Step 6: Create an Observability Helper

Before testing our agent, let's create a helper function that gives us **detailed visibility** into what the agent is doing.

**Why Observability Matters:**
- 🔍 See which tools the agent chooses
- 📊 Inspect the inputs passed to each tool
- ✅ Verify the outputs returned
- 🐛 Debug issues when the agent doesn't behave as expected

The `run_agent_verbose()` function will:
1. Display the user's query
2. Stream events as the agent works
3. Print details about each tool call
4. Return the final response

This is crucial for understanding and debugging agent behavior!

In [6]:
from llama_index.core.agent.workflow import ToolCallResult


async def run_agent_verbose(query: str):
    """Run the agent and print detailed information about tool calls."""
    print(f"🤖 User Query: {query}\n")
    print("=" * 60)

    handler = agent.run(query)
    async for event in handler.stream_events():
        if isinstance(event, ToolCallResult):
            print(f"🔧 Tool Used: {event.tool_name}")
            print(f"📥 Input: {event.tool_kwargs}")
            print(f"📤 Output: {event.tool_output}")
            print("-" * 60)

    result = await handler
    print(f"\n✨ Final Response:\n{result}\n")
    return result

## Step 7: Test Your Agent with Simple Queries

Let's start with basic queries to see how the agent uses tools:

In [7]:
response = await run_agent_verbose(
    "What's the weather like in Tokyo today?"
)

🤖 User Query: What's the weather like in Tokyo today?

🔧 Tool Used: get_weather
📥 Input: {'city': 'Tokyo', 'days_ahead': 0}
📤 Output: Weather in Tokyo on 2026-02-07: Cloudy, 29°C
------------------------------------------------------------

✨ Final Response:
The weather in Tokyo today is cloudy with a temperature of 29°C.



In [8]:
# Inspect which tools were called
print("Tool Calls Made:")
for tool_call in response.tool_calls:
    print(f"  - {tool_call}")

Tool Calls Made:
  - tool_name='get_weather' tool_kwargs={'city': 'Tokyo', 'days_ahead': 0} tool_id='call_0KVMsj0kh0Qz81ZrCNlkz3DR' tool_output=ToolOutput(blocks=[TextBlock(block_type='text', text='Weather in Tokyo on 2026-02-07: Cloudy, 29°C')], tool_name='get_weather', raw_input={'args': (), 'kwargs': {'city': 'Tokyo', 'days_ahead': 0}}, raw_output='Weather in Tokyo on 2026-02-07: Cloudy, 29°C', is_error=False) return_direct=False


## Step 8: Complex Multi-Tool Query

Watch how the agent chains multiple tools together to answer complex questions:

In [9]:
response = await run_agent_verbose(
    "I want to fly from London to Barcelona on 2025-12-15. What's the weather there, and how much is 500 USD in EUR?"
)

🤖 User Query: I want to fly from London to Barcelona on 2025-12-15. What's the weather there, and how much is 500 USD in EUR?

🔧 Tool Used: search_flights
📥 Input: {'origin': 'London', 'destination': 'Barcelona', 'date': '2025-12-15'}
📤 Output: Found flight FL835 from London to Barcelona on 2025-12-15. Price: $1463, Duration: 6h
------------------------------------------------------------
🔧 Tool Used: get_weather
📥 Input: {'city': 'Barcelona'}
📤 Output: Weather in Barcelona on 2026-02-07: Stormy, 23°C
------------------------------------------------------------
🔧 Tool Used: convert_currency
📥 Input: {'amount': 500, 'from_currency': 'USD', 'to_currency': 'EUR'}
📤 Output: 500 USD = 460.00 EUR
------------------------------------------------------------

✨ Final Response:
For your flight from London to Barcelona on 2025-12-15, there is a flight FL835 available with a price of $1463 and a duration of 6 hours.

The current weather in Barcelona is stormy with a temperature of 23°C.

Also, 50

## Step 9: Managing Context & Memory

By default, `.run()` is stateless (doesn't remember previous conversations). Let's add memory using a **Context** object:

In [10]:
from llama_index.core.workflow import Context

# Create a new agent instance for this demo
memory_agent = FunctionAgent(
    tools=[search_flights, get_weather, convert_currency],
    llm=llm
)

# Create a context to maintain conversation history
ctx = Context(memory_agent)

# First message - introduce yourself
response1 = await memory_agent.run(
    "Hi! I'm planning a trip to Rome in December. My name is Alex.",
    ctx=ctx
)
print(f"Response 1: {response1}\n")

# Second message - agent should remember your name
response2 = await memory_agent.run(
    "What's my name, and what's the weather like in my destination?",
    ctx=ctx
)
print(f"Response 2: {response2}")

Response 1: Hi Alex! That sounds like a wonderful trip. How can I assist you with your plans for Rome in December? Are you looking for flight options, weather information, or something else?

Response 2: Your name is Alex. The weather in Rome today is rainy with a temperature of around 20°C. If you want, I can also check the weather forecast for December or any specific date you have in mind.


## Step 10: Advanced - Model Context Protocol (MCP) Tools

MCP is a standardized protocol for connecting AI models to external tools and data sources. Let's integrate an MCP tool!

**What is MCP?**
- Open protocol for AI-to-tool communication
- Enables standardized tool integration
- Supports local and remote tools

### Creating a Custom MCP Tool

Let's create an MCP-compatible tool that fetches travel tips:

In [11]:
%pip install mcp httpx -q

In [12]:
from llama_index.core.tools import FunctionTool


def get_travel_tips(destination: str) -> str:
    """Get essential travel tips for a destination.

    Args:
        destination: The city or country to get tips for

    Returns:
        A string with helpful travel tips
    """
    tips_database = {
        "paris": "🇫🇷 Tips for Paris: Try croissants at local boulangeries, use the Metro, visit Eiffel Tower early morning",
        "tokyo": "🇯🇵 Tips for Tokyo: Get a Suica card for transport, try ramen at local shops, visit temples in Asakusa",
        "new york": "🇺🇸 Tips for NYC: Use subway, try pizza and bagels, visit Central Park and Times Square",
        "barcelona": "🇪🇸 Tips for Barcelona: Visit Sagrada Familia, try tapas and paella, walk Las Ramblas",
        "rome": "🇮🇹 Tips for Rome: Visit Colosseum early, try authentic carbonara, throw coin in Trevi Fountain",
    }

    city_lower = destination.lower()
    for key in tips_database:
        if key in city_lower:
            return tips_database[key]

    return f"💡 General tips: Research local customs, learn basic phrases, try local cuisine!"


# Convert to LlamaIndex tool
travel_tips_tool = FunctionTool.from_defaults(fn=get_travel_tips)

# Create agent with MCP-style tool
mcp_agent = FunctionAgent(
    tools=[search_flights, get_weather, convert_currency, travel_tips_tool],
    llm=llm
)

print("✅ Agent with MCP tool created!")

✅ Agent with MCP tool created!


### Testing the Enhanced Agent

In [13]:
async def run_mcp_agent(query: str):
    """Run the MCP-enhanced agent."""
    print(f"🤖 Query: {query}\n")
    handler = mcp_agent.run(query)
    async for event in handler.stream_events():
        if isinstance(event, ToolCallResult):
            print(f"🔧 {event.tool_name} → {event.tool_output}")

    result = await handler
    print(f"\n✨ {result}\n")
    return result


response = await run_mcp_agent(
    "I'm planning a trip to Tokyo. Give me travel tips and check the weather for tomorrow."
)

🤖 Query: I'm planning a trip to Tokyo. Give me travel tips and check the weather for tomorrow.

🔧 get_travel_tips → 🇯🇵 Tips for Tokyo: Get a Suica card for transport, try ramen at local shops, visit temples in Asakusa
🔧 get_weather → Weather in Tokyo on 2026-02-08: Partly Cloudy, 15°C

✨ Here are some travel tips for Tokyo: Get a Suica card for convenient transport, try ramen at local shops, and visit temples in Asakusa.

The weather in Tokyo tomorrow is expected to be partly cloudy with a temperature around 15°C. If you need more information or help with your trip, feel free to ask!



## Step 11: Enhanced Observability with Custom Callbacks

Let's implement more detailed observability to track agent performance:

In [14]:
import time
from typing import List


class AgentObserver:
    """Track and display agent performance metrics."""

    def __init__(self):
        self.tool_calls = []
        self.start_time = None
        self.end_time = None

    async def observe_agent(self, query: str, agent: FunctionAgent):
        """Run agent and collect observability data."""
        self.tool_calls = []
        self.start_time = time.time()

        print(f"🎯 Query: {query}\n")
        print("📊 Observability Dashboard")
        print("=" * 60)

        handler = agent.run(query)
        async for event in handler.stream_events():
            if isinstance(event, ToolCallResult):
                self.tool_calls.append({
                    'tool': event.tool_name,
                    'input': event.tool_kwargs,
                    'output': event.tool_output
                })
                print(f"⚙️  Tool Call #{len(self.tool_calls)}: {event.tool_name}")

        result = await handler
        self.end_time = time.time()

        # Display metrics
        print("\n" + "=" * 60)
        print("📈 Performance Metrics:")
        print(f"  ⏱️  Total Time: {self.end_time - self.start_time:.2f}s")
        print(f"  🔧 Tools Called: {len(self.tool_calls)}")
        print(f"  📝 Tool Names: {[t['tool'] for t in self.tool_calls]}")
        print(f"\n💬 Response: {result}")

        return result


# Test the observer
observer = AgentObserver()
await observer.observe_agent(
    "Find flights from Paris to Tokyo on 2025-12-20, check weather, and convert 1000 EUR to JPY",
    mcp_agent
)

🎯 Query: Find flights from Paris to Tokyo on 2025-12-20, check weather, and convert 1000 EUR to JPY

📊 Observability Dashboard
⚙️  Tool Call #1: search_flights
⚙️  Tool Call #2: get_weather
⚙️  Tool Call #3: convert_currency

📈 Performance Metrics:
  ⏱️  Total Time: 2.77s
  🔧 Tools Called: 3
  📝 Tool Names: ['search_flights', 'get_weather', 'convert_currency']

💬 Response: Here is the information you requested:

- Flight from Paris to Tokyo on 2025-12-20: Flight FL923, Price: $1428, Duration: 14 hours.
- Weather in Tokyo today: Partly Cloudy, 20°C.
- Currency conversion: 1000 EUR = 1000.00 JPY.

If you need any more details or assistance, feel free to ask!


AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='Here is the information you requested:\n\n- Flight from Paris to Tokyo on 2025-12-20: Flight FL923, Price: $1428, Duration: 14 hours.\n- Weather in Tokyo today: Partly Cloudy, 20°C.\n- Currency conversion: 1000 EUR = 1000.00 JPY.\n\nIf you need any more details or assistance, feel free to ask!')]), structured_response=None, current_agent_name='Agent', raw={'id': 'chatcmpl-D6agmbfrGbJtolqHewIzHhZUt7k3i', 'choices': [{'delta': {'content': None, 'function_call': None, 'refusal': None, 'role': None, 'tool_calls': None}, 'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'content_filter_results': {}}], 'created': 1770463372, 'model': 'gpt-4.1-mini-2025-04-14', 'object': 'chat.completion.chunk', 'service_tier': None, 'system_fingerprint': 'fp_3dcd5944f5', 'usage': None, 'obfuscation': 'VlTQKQvHLd'}, tool_calls=[ToolCallResult(tool_name='search_fligh

## 🎉 Congratulations!

You've learned how to:
- ✅ Create custom tools for AI agents
- ✅ Build functional agents with LlamaIndex
- ✅ Implement observability and debugging
- ✅ Manage conversation context and memory
- ✅ Integrate MCP-style tools
- ✅ Track agent performance metrics

### Share Your Work:
- Complete the challenge and share your restaurant agent
- Experiment with different tools and domains
- Try integrating real APIs for production use

Happy building! 🚀

## 🎓 Challenge: Build Your Own Agent!

Now it's your turn! Apply what you've learned to create a unique agent.

### Your Mission:
Create a **Restaurant Recommendation Agent** that helps users find and learn about restaurants.

### Requirements:
1. **Create at least 3 custom tools:**
   - `find_restaurants(cuisine: str, city: str) -> str`: Find restaurants by cuisine type
   - `get_restaurant_details(restaurant_name: str) -> str`: Get details like hours, price range
   - `make_reservation(restaurant_name: str, date: str, party_size: int) -> str`: Simulate booking

2. **Add a bonus tool** (get creative!):
   - Menu translator
   - Dietary restriction filter
   - Restaurant reviews summarizer
   - Distance calculator
   - Or your own idea!

3. **Implement:**
   - Agent with all your tools
   - Observability (print tool calls)
   - Context/memory for conversation
   - Test with at least 2 complex queries

4. **Bonus Points:**
   - Make it work with real APIs (like Yelp or Google Places)
   - Add error handling
   - Create a user-friendly interface
   - Implement caching for repeated queries

### Tips:
- Use clear docstrings - the LLM reads them!
- Test each tool individually first
- Start simple, then add complexity
- Use type hints for better tool understanding
- Think about real-world use cases

### Example Test Queries:
- "Find Italian restaurants in Boston and get details about the top one"
- "I'm vegetarian. Find a good restaurant in Seattle and make a reservation for 4 people tomorrow"
- "Recommend a restaurant with outdoor seating, check if they're open now, and book a table"

### Your Solution Space

Write your code below:

In [ ]:
# TODO: Define your restaurant tools here
def find_restaurants(cuisine: str, city: str) -> str:
    """Your implementation here"""
    pass


def get_restaurant_details(restaurant_name: str) -> str:
    """Your implementation here"""
    pass


def make_reservation(restaurant_name: str, date: str, party_size: int) -> str:
    """Your implementation here"""
    pass


# TODO: Add your bonus tool


# TODO: Create your agent
# restaurant_agent = FunctionAgent(...)


# TODO: Test your agent with complex queries
# response = await restaurant_agent.run("Your test query here")